In [2]:
import plotly.express as px
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import glob
import json

ModuleNotFoundError: No module named 'plotly'

In [3]:
%load_ext autoreload
%autoreload 2

import RDlite.accessor as ac

In [4]:
from RDlite import all_features

In [5]:
all_features

['company',
 'collection',
 'genre',
 'keyword',
 'rating',
 'language',
 'country',
 'name',
 'character',
 'gender',
 'movie',
 'adult',
 'budget',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'release_date',
 'revenue',
 'runtime',
 'status',
 'video',
 'vote_average',
 'vote_count']

In [6]:
ac.agg(*all_features, peek=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/themaster/workspace/cse314final/Data/companies.csv'